In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

2024-12-21 16:08:16.069347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734815296.081175  224909 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734815296.084707  224909 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 16:08:16.097323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import tensorflow as tf
print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))


GPUs disponibles: []


In [4]:
# Cargar el dataset
dataset = pd.read_csv('train.csv')

# Separar clases
class_majority = dataset[dataset['CHD_OR_MI'] == 1]
class_minority = dataset[dataset['CHD_OR_MI'] == 0]

# Undersampling de la clase mayoritaria para igualarla a la minoritaria
majority_downsampled = resample(class_majority, 
                                replace=False,  # No permite repetición
                                n_samples=len(class_minority), 
                                random_state=42)

# Crear un nuevo dataset balanceado
balanced_dataset = pd.concat([majority_downsampled, class_minority])

# Mezclar las filas del dataset
balanced_dataset = balanced_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

In [8]:
X_train_balanced.isnull().sum().sort_values(ascending=False)

SEX                    0
AGE                    0
HEALTH                 0
PHYSICAL_HEALTH        0
MENTAL_HEALTH          0
PHYSICAL_ACTIVITIES    0
BLOOD_PRESSURE         0
HIGH_CHOLESTEROL       0
SKIN_CANCER            0
CANCER                 0
BRONCHITIS             0
DEPRESSIVE_DISORDER    0
KIDNEY_DISEASE         0
DIABETES               0
ARTHRITIS              0
BMI                    0
DIFFICULTY_WALKING     0
SMOKE                  0
TABACCO_PRODUCTS       0
E_CIGARETTES           0
HEAVY_DRINKERS         0
HIV                    0
FRUITS                 0
VEGETABLES             0
FRIED_POTATOES         0
ETHNICITY              0
dtype: int64

In [5]:
# Dividir en predictores y objetivo
X_balanced = balanced_dataset.drop(columns=['CHD_OR_MI', 'ID'])
y_balanced = balanced_dataset['CHD_OR_MI']

# Imputar valores nulos
for col in X_balanced.columns:
    if X_balanced[col].dtype in ['float64', 'int64']:
        X_balanced[col].fillna(X_balanced[col].median(), inplace=True)
    else:
        X_balanced[col].fillna(X_balanced[col].mode()[0], inplace=True)

# Identificar y tratar outliers con capping
for col in X_balanced.select_dtypes(include=['float64', 'int64']).columns:
    Q1 = X_balanced[col].quantile(0.25)
    Q3 = X_balanced[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    X_balanced[col] = np.clip(X_balanced[col], lower_bound, upper_bound)

# Dividir en conjuntos de entrenamiento y prueba
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)


/tmp/ipykernel_224909/2695477708.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_balanced[col].fillna(X_balanced[col].median(), inplace=True)
/tmp/ipykernel_224909/2695477708.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [9]:
# Escalar los datos
scaler_balanced = StandardScaler()
X_train_scaled = scaler_balanced.fit_transform(X_train_balanced)
X_test_scaled = scaler_balanced.transform(X_test_balanced)

# Crear y compilar el modelo MLP
balanced_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

balanced_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Configurar EarlyStopping
early_stopping_balanced = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

/home/sergio/Documents/AI/KaggleNotebook/prediccion-de-sufrir-enfermedades-coronarias/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


InternalError: cudaSetDevice() on GPU:0 failed. Status: CUDA-capable device(s) is/are busy or unavailable

In [9]:
# Entrenar el modelo
history_balanced = balanced_model.fit(
    X_train_scaled, y_train_balanced,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping_balanced]
)



Epoch 1/50
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7453 - loss: 0.5202 - val_accuracy: 0.7647 - val_loss: 0.4948
Epoch 2/50
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7604 - loss: 0.4977 - val_accuracy: 0.7613 - val_loss: 0.5031
Epoch 3/50
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7656 - loss: 0.4903 - val_accuracy: 0.7606 - val_loss: 0.4940
Epoch 4/50
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7667 - loss: 0.4908 - val_accuracy: 0.7642 - val_loss: 0.4951
Epoch 5/50
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7687 - loss: 0.4877 - val_accuracy: 0.7594 - val_loss: 0.4986
Epoch 6/50
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7690 - loss: 0.4892 - val_accuracy: 0.7637 - val_loss: 0.4958
Epoch 7/50
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7688 - loss: 0.4891 - val_accuracy: 0.7641 - val_loss: 0.4944
Epoch 8/50
1131/1131 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7694 - loss: 0.4841 - 

In [10]:
# Evaluar el modelo
y_pred_prob_balanced = balanced_model.predict(X_test_scaled)
y_pred_balanced = (y_pred_prob_balanced > 0.5).astype(int)

# Calcular métricas
f1_balanced = f1_score(y_test_balanced, y_pred_balanced)
accuracy_balanced = accuracy_score(y_test_balanced, y_pred_balanced)

print(f"F1 Score: {f1_balanced}")
print(f"Accuracy: {accuracy_balanced}")


354/354 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
F1 Score: 0.747254809948381
Accuracy: 0.761765746638358


In [17]:
# Dividir en predictores y objetivo
test_dataset = pd.read_csv('test_public.csv')

x_test = test_dataset.drop(columns=['CHD_OR_MI', 'ID'])
y_test = test_dataset['CHD_OR_MI']

# Imputar valores nulos
for col in x_test.columns:
    if x_test[col].dtype in ['float64', 'int64']:
        x_test[col].fillna(x_test[col].median(), inplace=True)
    else:
        x_test[col].fillna(x_test[col].mode()[0], inplace=True)

# Identificar y tratar outliers con capping
for col in x_test.select_dtypes(include=['float64', 'int64']).columns:
    Q1 = x_test[col].quantile(0.25)
    Q3 = x_test[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    x_test[col] = np.clip(x_test[col], lower_bound, upper_bound)

# Escalar los datos
scaler_balanced = StandardScaler()
x_test = scaler_balanced.fit_transform(x_test)
x_test = scaler_balanced.transform(x_test)


# Evaluar el modelo
y_pred_prob_balanced = balanced_model.predict(x_test)

# Calcular métricas
f1_balanced = f1_score(y_test, (y_pred_prob_balanced > 0.5).astype(int))
accuracy_balanced = accuracy_score(y_test, (y_pred_prob_balanced > 0.5).astype(int))
report = classification_report(y_test, (y_pred_prob_balanced > 0.5).astype(int))
# Imprimir métricas

print(f"F1 Score: {f1_balanced}")
print(f"Accuracy: {accuracy_balanced}")
print(report)
    



 103/1357 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step

/tmp/ipykernel_36162/2492555409.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_test[col].fillna(x_test[col].median(), inplace=True)
/tmp/ipykernel_36162/2492555409.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

1357/1357 ━━━━━━━━━━━━━━━━━━━━ 1s 529us/step
F1 Score: 0.780552023337572
Accuracy: 0.6620513293093121
              precision    recall  f1-score   support

         0.0       0.16      0.75      0.27      3532
         1.0       0.97      0.65      0.78     39874

    accuracy                           0.66     43406
   macro avg       0.56      0.70      0.52     43406
weighted avg       0.90      0.66      0.74     43406

